# `rules` are magic functions and regular expressions that express unique displays from strings.

In [1]:
import abc, fnmatch, IPython, functools

In [2]:
import vdom; from vdom import div, img; from vdom.svg import iframe
from mimetypes import guess_type; guess = lambda x: guess_type(x)[0]

In [3]:
     import IPython

In [4]:
    from dataclasses import dataclass

In [5]:
    import inspect

In [6]:
    import operator
    import string

In [7]:
    @dataclass
    class NoArgsCellMagic:
        callable: callable = None
        __name__ = property(operator.attrgetter('callable.__name__'))
        __doc__ = property(operator.attrgetter('callable.__doc__'))
        def __call__(self, line, cell): return self.callable(cell)

In [8]:
    import mistune

In [9]:
    class MagicStringsDispatch(abc.ABCMeta):
        
        def append(self, key):
            ip = IPython.get_ipython()
            def attach(callable): 
                ip.register_magic_function(NoArgsCellMagic(callable), 'cell')
                setattr(self, key, staticmethod(callable))
                return callable
            return attach

In [10]:
    from dataclasses import dataclass, field

In [14]:
    import IPython
    from pidgin import weave

In [15]:
    def none(object): return

In [16]:
    @dataclass
    class MagicStrings(metaclass=MagicStringsDispatch):
        ip: IPython.core.interactiveshell.InteractiveShell = field(default_factory=IPython.get_ipython, repr=False)
        namespace: dict = field(default_factory=dict, repr=False)
        previous: callable = field(default=none)
        def __post_init__(self):
            self.namespace = self.namespace or self.ip.user_ns
            
        def __call__(self, str, result=None):
            from .environment import IPythonDollarTemplate
            str = IPythonDollarTemplate(str).safe_substitute(**self.ip.user_ns)
            for attr in (x for x in dir(self) if '*' in x):
                if fnmatch.fnmatch(str, attr):
                    self.ip.log.error((self, attr))
                    result = getattr(self, attr)(str)
                    
                    if isinstance(result, dict) and str in result:
                        result = result[str]
                        if result['status'] == 'ok':
                            return result['data'], result['metadata']
                        else: ...
                            
                    elif isinstance(result, type(str)):
                        str = result
                        continue

                    elif result is not None:
                        data, metadata = IPython.get_ipython().display_formatter.format(result)
                        return data, metadata
                    
                
            if str and str.splitlines()[0].strip():
                # IPython.display.display(weave.Document(str, should_display=False, template=False))
                return {
                    'text/markdown': str
                }, {}
            
        def __enter__(self):
            try: self.previous = self.ip.display_formatter.mimebundle_formatter.lookup_by_type(str)
            except: self.previous = None
            finally: self.ip.display_formatter.mimebundle_formatter.for_type(str, self)
            
        def __exit__(self, *exc): self.ip.display_formatter.mimebundle_formatter.for_type(str, self.previous)

In [ ]:
@MagicStrings.append(',*')
@MagicStrings.append(';*')
def eval_shorthand_ipython(str):
    ip = IPython.get_ipython()
    return eval(ip.input_transformer_manager.transform_cell(str), ip.user_ns, ip.user_global_ns)

In [ ]:
def load_ipython_extension(ip): MagicStrings().__enter__()
    
def unload_ipython_extension(ip): MagicStrings().__exit__()
    
__name__ == '__main__' and load_ipython_extension(get_ipython())